# Major-TOM Filtering

In [1]:
from pathlib import Path

SOURCE_DATASET = 'Major-TOM/Core-S2L2A' # Identify HF Dataset
DATASET_DIR = Path('./data/Major-TOM/')
DATASET_DIR.mkdir(exist_ok=True, parents=True)
ACCESS_URL = 'https://huggingface.co/datasets/{}/resolve/main/metadata.parquet?download=true'.format(SOURCE_DATASET)
LOCAL_URL = DATASET_DIR / '{}.parquet'.format(ACCESS_URL.split('.parquet')[0].split('/')[-1])

In [2]:
import urllib.request
LOCAL_URL, RESPONSE = urllib.request.urlretrieve(ACCESS_URL, LOCAL_URL)

In [3]:
import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd

df = pq.read_table(LOCAL_URL).to_pandas()
df['timestamp'] = pd.to_datetime(df.timestamp)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.centre_lon, df.centre_lat), crs=df.crs.iloc[0]
)
gdf.head()

,grid_cell,grid_row_u,grid_col_r,product_id,timestamp,cloud_cover,nodata,centre_lat,centre_lon,crs,parquet_url,parquet_row,geometry
0,922D_249L,-922,-249,S2A_MSIL2A_20230119T161811_N0509_R111_T01CDJ_2...,2023-01-19 16:18:11,18.941737,0.0,-82.770666,-178.200331,EPSG:32701,https://huggingface.co/datasets/Major-TOM/Core...,0,POINT (-178.200 -82.771)
1,922D_245L,-922,-245,S2B_MSIL2A_20181219T162339_N9999_R011_T01CEJ_2...,2018-12-19 16:23:39,22.742201,0.0,-82.768451,-175.349546,EPSG:32701,https://huggingface.co/datasets/Major-TOM/Core...,1,POINT (-175.350 -82.768)
2,922D_244L,-922,-244,S2A_MSIL2A_20200119T155811_N9999_R025_T01CEJ_2...,2020-01-19 15:58:11,0.000000,0.0,-82.767914,-174.636985,EPSG:32701,https://huggingface.co/datasets/Major-TOM/Core...,2,POINT (-174.637 -82.768)
3,922D_243L,-922,-243,S2A_MSIL2A_20210103T155811_N9999_R025_T01CEJ_2...,2021-01-03 15:58:11,3.769691,0.0,-82.767385,-173.924477,EPSG:32701,https://huggingface.co/datasets/Major-TOM/Core...,3,POINT (-173.924 -82.767)
4,922D_242L,-922,-242,S2B_MSIL2A_20181220T155319_N9999_R025_T01CEJ_2...,2018-12-20 15:53:19,0.000000,0.0,-82.766864,-173.212021,EPSG:32701,https://huggingface.co/datasets/Major-TOM/Core...,4,POINT (-173.212 -82.767)


In [6]:
def filter_metadata(df,
                    region=None,
                    daterange=None,
                    cloud_cover=(0,100),
                    nodata=(0, 1.0)
                   ):
    """Filters the Major-TOM dataframe based on several parameters

    Args:
        df (geopandas dataframe): Parent dataframe
        region (shapely geometry object) : Region of interest
        daterange (tuple) : Inclusive range of dates (example format: '2020-01-01')
        cloud_cover (tuple) : Inclusive percentage range (0-100) of cloud cover
        nodata (tuple) : Inclusive fraction (0.0-1.0) of no data allowed in a sample

    Returns:
        df: a filtered dataframe
    """
    # temporal filtering
    if daterange is not None:
        assert (isinstance(daterange, list) or isinstance(daterange, tuple)) and len(daterange)==2
        df = df[df.timestamp >= daterange[0]]
        df = df[df.timestamp <= daterange[1]]
    
    # spatial filtering
    if region is not None:
        idxs = df.sindex.query(region)
        df = df.take(idxs)
    # cloud filtering
    if cloud_cover is not None:
        df = df[df.cloud_cover >= cloud_cover[0]]
        df = df[df.cloud_cover <= cloud_cover[1]]

    # spatial filtering
    if nodata is not None:
        df = df[df.nodata >= nodata[0]]
        df = df[df.nodata <= nodata[1]]

    return df

In [4]:
from shapely.geometry import box

# Example bounding boxes used for filtering
switzerland = box(5.9559111595,45.8179931641,10.4920501709,47.808380127)
gabon = box(8.1283659854,-4.9213919841,15.1618722208,2.7923006325)
napoli = box(14.091710578,40.7915558593,14.3723765416,40.9819258062)
pacific = box(-153.3922893485,39.6170415622,-152.0423077748,40.7090892316) # a remote patch over pacific - no data

In [7]:
filtered_df = filter_metadata(gdf,
                              cloud_cover = (0,10),
                              region=switzerland,
                              daterange=('2020-01-01', '2025-01-01'),
                              nodata=(0.0,0.0)
                              )

len(filtered_df)

368

In [8]:
from fsspec.parquet import open_parquet_file
import pyarrow.parquet as pq
from io import BytesIO
from PIL import Image
from rasterio.io import MemoryFile

def read_row(row, columns=["thumbnail"]):
    """Reads a row from a Major-TOM dataframe

    Args:
        row (row from geopandas dataframe): The row of metadata
        columns (list): columns to be read from the file

    Returns:
        data (dict): dictionary with returned data from requested columns
    """
    with open_parquet_file(row.parquet_url,columns = columns) as f:
        with pq.ParquetFile(f) as pf:
            row_group = pf.read_row_group(row.parquet_row, columns=columns)

    if columns == ["thumbnail"]:
        stream = BytesIO(row_group['thumbnail'][0].as_py())
        return Image.open(stream)
    else:
        row_output = {}
        for col in columns:
            bytes = row_group[col][0].as_py()

            if col != 'thumbnail':
                with MemoryFile(bytes) as mem_f:
                    with mem_f.open(driver='GTiff') as f:
                        row_output[col] = f.read().squeeze()
            else:
                stream = BytesIO(bytes)
                row_output[col] = Image.open(stream)

        return row_output
        

out = read_row(filtered_df.iloc[0], columns = ['B04', 'thumbnail'])

### Download

In [9]:
filtered_df

,grid_cell,grid_row_u,grid_col_r,product_id,timestamp,cloud_cover,nodata,centre_lat,centre_lon,crs,parquet_url,parquet_row,geometry
1697089,510U_80R,510,80,S2B_MSIL2A_20211018T101939_N0500_R065_T32TPR_2...,2021-10-18 10:19:39,0.000000,0.0,45.852619,10.373241,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,176,POINT (10.373 45.853)
1697078,510U_69R,510,69,S2A_MSIL2A_20200521T102031_N0500_R065_T32TMR_2...,2020-05-21 10:20:31,1.069327,0.0,45.853430,8.954794,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,165,POINT (8.955 45.853)
1697077,510U_68R,510,68,S2B_MSIL2A_20211018T101939_N0500_R065_T32TMR_2...,2021-10-18 10:19:39,0.000000,0.0,45.853502,8.825842,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,164,POINT (8.826 45.854)
1697076,510U_67R,510,67,S2A_MSIL2A_20201028T102141_N0500_R065_T32TMR_2...,2020-10-28 10:21:41,0.000000,0.0,45.853574,8.696890,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,163,POINT (8.697 45.854)
1697074,510U_65R,510,65,S2A_MSIL2A_20201028T102141_N0500_R065_T32TMR_2...,2020-10-28 10:21:41,0.000000,0.0,45.853718,8.438985,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,161,POINT (8.439 45.854)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1736201,531U_48R,531,48,S2A_MSIL2A_20230820T103631_N0509_R008_T32TLT_2...,2023-08-20 10:36:31,0.000000,0.0,47.741042,6.469137,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,290,POINT (6.469 47.741)
1736219,531U_66R,531,66,S2A_MSIL2A_20210903T102021_N0301_R065_T32TMT_2...,2021-09-03 10:20:21,0.000000,0.0,47.739692,8.873050,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,308,POINT (8.873 47.740)
1736218,531U_65R,531,65,S2A_MSIL2A_20200401T102021_N0500_R065_T32TMT_2...,2020-04-01 10:20:21,0.000000,0.0,47.739769,8.739502,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,307,POINT (8.740 47.740)
1736216,531U_63R,531,63,S2A_MSIL2A_20230625T101601_N0509_R065_T32TMT_2...,2023-06-25 10:16:01,0.000000,0.0,47.739922,8.472406,EPSG:32632,https://huggingface.co/datasets/Major-TOM/Core...,305,POINT (8.472 47.740)


In [12]:
from tqdm import tqdm
import os
import fsspec

def download_parquet(df, local_dir, source_name, url, temp_filestub, by_row=False, tif_columns=None):
    # Turn for loop of filter_download into parallelizable func

    # identify all relevant rows
    rows = df[df.parquet_url == url].parquet_row.unique()
    
    if not by_row: # (downloads entire parquet)        
        # download a temporary file
        temp_file = temp_filestub+url.split('/')[-1]
        temp_path, http_resp = urllib.request.urlretrieve(url, temp_file)            
    else:
        f=fsspec.open(url)
        temp_path = f.open()

    # populate the bands
    with pq.ParquetFile(temp_path) as pf:
        for row_idx in rows:
            table = pf.read_row_group(row_idx)

            product_id = table['product_id'][0].as_py()
            grid_cell = table['grid_cell'][0].as_py()
            row = grid_cell.split('_')[0]
        
            dest = local_dir / Path("{}/{}/{}/{}".format(source_name, row, grid_cell, product_id))
            dest.mkdir(exist_ok=True, parents=True)

            columns = [col for col in table.column_names if col[0] == 'B'] + ['cloud_mask'] if tif_columns is None else tif_columns
            # tifs
            for col in columns:
                with open(dest / "{}.tif".format(col), "wb") as f:
                    # Write bytes to file
                    f.write(table[col][0].as_py())

            # thumbnail (png)
            col = 'thumbnail'
            with open(dest / "{}.png".format(col), "wb") as f:
                # Write bytes to file
                f.write(table[col][0].as_py())
    if not by_row:
        # remove downloaded file
        os.remove(temp_path)
    else:
        f.close()


def filter_download(df, local_dir, source_name, by_row=False, verbose=False, tif_columns=None, num_workers=1):
    """Downloads and unpacks the data of Major-TOM based on a metadata dataframe

    Args:
        df (geopandas dataframe): Metadata dataframe
        local_dir (str or Path) : Path to the where the data is to be stored locally
        source_name (str) : Name alias of the resulting dataset
        num_workers (int) : Number of workers for parallel download
        by_row (bool): If True, it will access individual rows of parquet via http - otherwise entire parquets are downloaded temporarily
        verbose (bool) : option for potential internal state printing


    Returns:
        None

    """

    if isinstance(local_dir, str):
        local_dir = Path(local_dir)

    temp_filestub = local_dir / 'temp_'

    # identify all parquets that need to be downloaded (group them)
    urls = df.parquet_url.unique()
    print('Starting download of {} parquet files.'.format(len(urls))) if verbose else None

    if num_workers > 1:
        from multiprocessing import Pool
        pool = Pool(num_workers)
        pool.starmap(download_parquet, [(df, local_dir, source_name, url, temp_filestub, by_row, tif_columns) for url in urls])
        pool.close()
    else:
        for url in tqdm(urls, desc='Downloading and unpacking...'):
            download_parquet(df, local_dir, source_name, url, temp_filestub, by_row, tif_columns)

In [13]:
filter_download(filtered_df, local_dir='./data/swizz', source_name='L2A', by_row=True,num_workers=2)

['https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03395.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03399.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03402.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03406.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03410.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03413.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03414.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03417.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03421.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S2L2A/resolve/main/images/part_03425.parquet'
 'https://huggingface.co/datasets/Major-TOM/Core-S

KeyboardInterrupt: 

In [150]:
filter_download(filtered_df, local_dir='./data/mczerkawski', source_name='L2A', by_row=False)

In [17]:
import os
import pandas as pd
from torch.utils.data import Dataset
from pathlib import Path
import rasterio as rio

class MajorTOM(Dataset):
    """MajorTOM Dataset (https://huggingface.co/Major-TOM)

    Args:
        df ((geo)pandas.DataFrame): Metadata dataframe
        local_dir (string): Root directory of the local dataset version
        tif_bands (list): A list of tif file names to be read
        png_bands (list): A list of png file names to be read
        
    """
    
    def __init__(self,
                 df,
                 local_dir = None,
                 tif_bands=['B04','B03','B02'],
                 png_bands=['thumbnail']
                ):
        super().__init__()
        self.df = df
        self.local_dir = Path(local_dir) if isinstance(local_dir,str) else local_dir
        self.tif_bands = tif_bands if not isinstance(tif_bands,str) else [tif_bands]
        self.png_bands = png_bands if not isinstance(png_bands,str) else [png_bands]
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        meta = self.df.iloc[idx]

        product_id = meta.product_id
        grid_cell = meta.grid_cell
        row = grid_cell.split('_')[0]
    
        path = self.local_dir / Path("{}/{}/{}".format(row, grid_cell, product_id))
        out_dict = {'meta' : meta}
        
        for band in self.tif_bands:
            with rio.open(path / '{}.tif'.format(band)) as f:
                out = f.read()
            out_dict[band] = out

        for band in self.png_bands:
            out_dict[band] = Image.open(path / '{}.png'.format(band))

        return out_dict
        
ds = MajorTOM(filtered_df, './data/mczerkawski/L2A')

ds[0]

{'meta': grid_cell                                              454U_120R
 grid_row_u                                                   454
 grid_col_r                                                   120
 product_id     S2B_MSIL1C_20220719T095559_N0400_R122_T33TVF_2...
 timestamp                                    2022-07-19 09:55:59
 cloud_cover                                                  0.0
 nodata                                                       0.0
 centre_lat                                             40.823861
 centre_lon                                             14.292709
 crs                                                   EPSG:32633
 parquet_url    https://huggingface.co/datasets/Major-TOM/Core...
 parquet_row                                                  455
 geometry             POINT (14.29270867998482 40.82386055223662)
 Name: 1593374, dtype: object,
 'cloud_mask': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..